In [13]:
import csv, re
import pandas as pd
import json

In [79]:
vcffile = '../tests/vcfs/test.vcf'
#vcffile = '../tests/vcfs/010422/Oncomine_Variants_(5.16)_filtered.vcf'

def parse_thermo_vcf(vcf):
    ''' Les inn vcf til pandas dataframe'''
    df = pd.read_csv(vcffile, sep="\t", comment='#', names=["CHROM","POS","ID","REF","ALT","QUAL","FILTER","INFO","FORMAT","GT"])
    return df
   
def filter_nocalls(df):
    ''' Fjern varianter som begynner med 0/0 eller ./.'''
    dfOut = df[   (~(df['GT'].str.startswith('./.')))   & (~(df['GT'].str.startswith('0/0')))   ]
    return dfOut

def explode_format_gt(df):
    ''' Eksploderer GT OG FORMAT-kolonnene til egne kolonner
    Følgende kolonner finnes to steder, så de fjernes fra format: AF,AO,DP,FAO,FDP,FRO,FSAF,FSAR,FSRF,FSRR,RO,SAF,SAR,SRF,SRR'''
    df.reset_index(inplace=True,drop=True)
    ny = pd.DataFrame(list(dict(zip(a,b)) for a,b in zip(df['FORMAT'].str.split(":"), df['GT'].str.split(":"))))
    for i in ["AF","AO","DP","FAO","FDP","FRO","FSAF","FSAR","FSRF","FSRR","RO","SAF","SAR","SRF","SRR"]:
        try:
            ny = ny.drop(i, axis=1)
        except:
            pass
    dfOut = pd.concat([df, ny], axis=1)
    del dfOut["GT"]
    del dfOut["FORMAT"]
    return dfOut


# Gjerne skriv disse tre funksjonene som en funksjon, med input:
def replace_semi(inputstring):
    '''  Hvor det finnes semikolon inne i curly braces i INFO-kolonnen gjoeres denne om til underscore - ellers blir det feilmelding'''
    return re.sub(r"{[^{}]+}", lambda x: x.group(0).replace(";", "_"), inputstring)

def replace_comma(inputstring):
    ''' replaces , inside ' with | '''
    inputstring = inputstring.replace("[{", "").replace("}]", "")#.replace("[{", "").replace("}]", "")
    
    return re.sub(r"(?!(([^']*'){2})*[^']*$),", '|', inputstring)

def replace_comma_2(inputstring):
    ''' replace comma inne i [] med |  '''
    return re.sub(r"[^[]+]", lambda x: x.group(0).replace(",", "|"), inputstring)


def explode_info(df):
    ''' Eksploderer INFO-kolonnen til egne kolonner'''
    df.reset_index(inplace=True,drop=True)
    # First maa vi fjerne alle semikolon som er inne i klammer:
    df["INFO"] = df["INFO"].apply(replace_semi)
    
    # Replace HS MED HS=NA
    df['INFO'] = df['INFO'].str.replace(';HS;',';HS=NA;')
    ny2 = pd.DataFrame([dict(w.split('=', 1) for w in x) for x in df["INFO"].str.split(";")])
    dfOut = pd.concat([df, ny2], axis=1)
    del dfOut["INFO"]
    return dfOut

def explode_func(df):
    ''' Eksploderer FUNC-kolonnen til egne kolonner'''
    df.reset_index(inplace=True,drop=True)
    df["FUNC"] = df["FUNC"].apply(replace_comma)
    
    
    #df["FUNC"] = df["FUNC"].apply(replace_comma_2)
    #return df
    print df["FUNC"]
    # Sjekk om det finnes flere
    for x in df["FUNC"].str.split(","):
        print x
    #ny2 = pd.DataFrame([dict(w.split(':', 1) for w in x) for x in df["FUNC"].str.split(",")])
    #dfOut = pd.concat([df, ny2], axis=1)
    #del dfOut["FUNC"]
    #return dfOut
    
  #result = text.replace(text[0:3], replacement, 1)

In [80]:
df = parse_thermo_vcf(vcffile)
df = filter_nocalls(df)
df = explode_format_gt(df)

df = explode_info(df)
#df = explode_func(df)
#df = df.iloc[0:5]

In [98]:

def check_dict_duplicates(inputstr):
    ''' Sjekker om det er duplikate keys i FUNC. Typisk origPos kan vare duplisert'''




def f(input):
    x = input.replace("[{","{").replace("}]","}")
    x = x.replace("\"", "'").replace("\'","\"")
    
    
    # Teste om det finnes flere versioner av noen av keysene - hvis ikke kan man gjøre om til json på direkten
    if len(re.split(r',(?![^{}]*\})', x)) == 1:
        pass
        #print json.loads(x)
    else:
        # hvis det er flere med samme navn, legge til _2 til duplikatet
        print x
        for i in re.split(r',(?![^{}]*\})', x):
            print i
        print ""
    
    
    
    
    
    
    return x
    
df["FUNC"].apply(f)

{"origPos":"120478285","origRef":"T","normalizedRef":"T","gene":"NOTCH2NLC","normalizedPos":"120478285","normalizedAlt":"C","gt":"pos","coding":"c.*7402+53590A>G","transcript":"NM_001364012.2","function":"unknown","protein":"p.?","location":"intronic","origAlt":"C"},{"origPos":"120478285","origRef":"T","normalizedRef":"T","gene":"NOTCH2","normalizedPos":"120478285","normalizedAlt":"C","gt":"pos","coding":"c.3523-58A>G","transcript":"NM_024408.4","function":"unknown","protein":"p.?","location":"intronic","origAlt":"C"}
{"origPos":"120478285","origRef":"T","normalizedRef":"T","gene":"NOTCH2NLC","normalizedPos":"120478285","normalizedAlt":"C","gt":"pos","coding":"c.*7402+53590A>G","transcript":"NM_001364012.2","function":"unknown","protein":"p.?","location":"intronic","origAlt":"C"}
{"origPos":"120478285","origRef":"T","normalizedRef":"T","gene":"NOTCH2","normalizedPos":"120478285","normalizedAlt":"C","gt":"pos","coding":"c.3523-58A>G","transcript":"NM_024408.4","function":"unknown","prot

0      {"origPos":"11187893","origRef":"T","normalize...
1      {"origPos":"27089638","origRef":"A","normalize...
2      {"origPos":"27100205","origRef":"A","normalize...
3      {"origPos":"40363054","origRef":"G","normalize...
4      {"origPos":"65311214","origRef":"G","normalize...
5      {"origPos":"120478285","origRef":"T","normaliz...
6      {"origPos":"120612006","origRef":"G","normaliz...
7      {"origPos":"120612040","origRef":"T","normaliz...
8      {"origPos":"156785617","origRef":"G","normaliz...
9      {"origPos":"123298158","origRef":"T","normaliz...
10     {"origPos":"69462910","origRef":"G","normalize...
11     {"origPos":"94197260","origRef":"A","normalize...
12     {"origPos":"94212048","origRef":"C","normalize...
13     {"origPos":"94225920","origRef":"C","normalize...
14     {"origPos":"108183167","origRef":"A","normaliz...
15     {"origPos":"108196712","origRef":"CATT","norma...
16     {"origPos":"108216415","origRef":"TTAGTTT","no...
17     {"origPos":"108225661","

In [100]:
import json
x = {"origPos":"120478285","origRef":"T","normalizedRef":"T","gene":"NOTCH2NLC","normalizedPos":"120478285","normalizedAlt":"C","gt":"pos","coding":"c.*7402+53590A>G","transcript":"NM_001364012.2","function":"unknown","protein":"p.?","location":"intronic","origAlt":"C"},{"origPos":"120478285","origRef":"T","normalizedRef":"T","gene":"NOTCH2","normalizedPos":"120478285","normalizedAlt":"C","gt":"pos","coding":"c.3523-58A>G","transcript":"NM_024408.4","function":"unknown","protein":"p.?","location":"intronic","origAlt":"C"}
x = x.replace("\"", "'").replace("\'","\"")
x
#x = '{"origPos":"11187893","origRef":"T","normalizedRef":"T","gene":"MTOR","normalizedPos":"11187893","normalizedAlt":"C","gt":"pos","coding":"c.6034-30A>G","transcript":"NM_004958.4","function":"unknown","protein":"p.?","location":"intronic","origAlt":"C"}'
#json.loads(x)

AttributeError: 'tuple' object has no attribute 'replace'

In [57]:
x = "'origPos':'11187893'","'origRef':'T'"
for i in x:
    
    print [j.replace("'","") for j in  i.split(":")]
    

['origPos', '11187893']
['origRef', 'T']
